In [70]:
import_statements()

NameError: name 'PiecewisePolynomial' is not defined

In [34]:
from itertools import combinations_with_replacement
from ehrhart_quasi_polynomial import ehrhart_quasi_polynomial
from sage.rings.polynomial.groebner_fan import PolyhedralFan

In [35]:
from sage.functions.other import factorial
from sage.modules.free_module_element import free_module_element
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing

In [36]:
from inspect import *

In [37]:
def create_polytope_from_matrix(A, b):
    inequalities = [[b[k]] + list(-A.rows()[k]) for k in range(A.nrows())]
    return Polyhedron(ieqs = inequalities)

In [38]:
def generate_cone_points(cone, number):
    dimension = cone.dimension()
    amb_dim = cone.ambient_dimension()

    # print(f"cone of dim = {dimension} in {amb_dim}d ambient space")

    if dimension == 0:
        return [(0,)*amb_dim], 0

    vectors = [free_module_element(list(ray)) for ray in cone.rays()]

    points = []
    points_len = 0
    index = 1
    while points_len <= number:
        new_points = [sum(combi) for combi in combinations_with_replacement(vectors, index)]
        points += new_points
        index += 1
        points_len += len(new_points)
    return points[:number]

In [39]:
def _gfan_input_style(A_matrix):
    return "{" + ", ".join(str(row) for row in A_matrix.rows()) + "}"

In [69]:
PolyhedralFan(gfan(_gfan_input_style(Matrix([[-1, 0], [0, 2]])), "secondaryfan")).house()

AttributeError: 'PolyhedralFan' object has no attribute 'house'

In [83]:
A = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
sec_fan = PolyhedralFan(gfan(_gfan_input_style(A), "secondaryfan"))
sec_fan.fan_dict["LINEALITY_SPACE"]

['1 0 -1 0', '0 1 -1 -1']

In [42]:
sec_fan.rays()

[[-1, 2, -1, 3], [1, 3, 1, 2], [2, 1, 2, -1]]

In [43]:
def get_terms_of_order(polynomials, order):
    return [poly.coefficients()[order].constants()[0] if poly.degree() >= order else 0 for poly in polynomials]

In [44]:
def get_term_dict(A_matrix, points):
    polynomials = []
    for point in points:
        polytope = create_polytope_from_matrix(A_matrix, point)
        ehr_poly = ehrhart_quasi_polynomial(polytope.Vrepresentation())
        polynomials.append(ehr_poly)

    max_degree = max(poly.degree() for poly in polynomials)
    terms = {d: get_terms_of_order(polynomials, d) for d in range(max_degree + 1)}
    return terms

In [45]:
def _remove_zero_dimensions(rays):
    zero_dimensions = []
    for dim, val in enumerate(rays[0]):
        if val == 0:
            if all(ray[dim] == 0 for ray in rays[1:]):
                zero_dimensions.append(dim)
    reduced_cone = Cone([[val for dim, val in enumerate(ray) if dim not in zero_dimensions ]for ray in rays])
    return reduced_cone, zero_dimensions

In [46]:
def _safe_ratio(numerator, denominator):
    return numerator/denominator if denominator != 0 else None

In [47]:
def _remove_scaled_dimensions(rays, ambient_dimension):
    scaled_dimensions = []
    for dim in range(ambient_dimension):
        for extra_dim in range(dim):
            ratio = _safe_ratio(rays[0][dim], rays[0][extra_dim])
            for ray in rays[1:]:
                if ratio != _safe_ratio(ray[dim], ray[extra_dim]):
                    break
            else:
                scaled_dimensions.append(extra_dim)

    reduced_cone = Cone([[val for dim, val in enumerate(ray) if dim not in scaled_dimensions] for ray in rays])
    return reduced_cone, scaled_dimensions

In [48]:
def _find_true_dimensions_removed(first, second, starting_dimension):
    dimensions_removed = []
    true_dimension = 0
    skipped_dimensions = 0
    while true_dimension < starting_dimension:
        if true_dimension in first:
            dimensions_removed.append(true_dimension)
            skipped_dimensions += 1
        if true_dimension - skipped_dimensions in second:
            dimensions_removed.append(true_dimension)
        true_dimension += 1
    return dimensions_removed

In [49]:
def _reduce_cone(cone):
    if cone.dimension() in [0, 1]:
        return cone, []
    ambient_dimension = cone.ambient_dimension()

    cone, zero_dimensions = _remove_zero_dimensions(cone.rays())
    return cone, zero_dimensions
    
    cone, scaled_dimensions = _remove_scaled_dimensions(cone.rays(), ambient_dimension - len(zero_dimensions))

    dimensions_removed = _find_true_dimensions_removed(zero_dimensions, scaled_dimensions, ambient_dimension)
    return cone, dimensions_removed

In [50]:
cone = Cone([(0, 1, 2, 4), (0, 0, 1, 2)])
print(cone.rays())
cone, removed = _reduce_cone(cone)
print(cone.rays(), removed)

N(0, 1, 2, 4),
N(0, 0, 1, 2)
in 4-d lattice N
N(1, 2, 4),
N(0, 1, 2)
in 3-d lattice N [0]


In [51]:
def _alter_rays(rays, alter, num_var):
    for key, val in alter.items():
            if key >= num_variables:
                raise ValueError(f"{keys=} of 'alter' cannot be larger or equal than the dimension of the fan, which is {sec_fan.dim()}")
            if val not in QQ:
                raise ValueError(f"values of 'alter' need to be rationals, got {val} of type {type(val)}")

    altered_rays = []
    for ray in fan_rays:
        altered_ray = ray
        for key, val in alter.items():
            altered_ray[key] = val
        altered_rays.append(altered_ray)
    return altered_rays

In [52]:
def _compute_piecewise(A_matrix, sec_fan, alter=None):
    num_variables = A_matrix.nrows()
    max_degree = sec_fan.lineality_dim()
    print(f"{num_variables=}")

    R = PolynomialRing(QQ, "x", num_variables)
    x_vars = R.gens()
    S = PolynomialRing(R, "k")
    k = S.gen()

    fan_rays = sec_fan.rays()

    if isinstance(alter, dict):
        fan_rays = _alter_rays(fan_rays, alter, num_variables)

    cone_polys = []
    for cone_dimension, ray_lists in sec_fan.maximal_cones().items():
        for ray_list in ray_lists:
            if ray_list == []: # only the origin is included
                poly_origin = create_polytope_from_matrix(A_matrix, [0]*num_variables)
                ehr_origin = ehrhart_quasi_polynomial(poly_origin.Vrepresentation())
                cone_polys.append(ehr_origin)
                continue

            cone = Cone([fan_rays[idx] for idx in ray_list])
            # estimated_period = 1
            red_cone, dimensions_removed = _reduce_cone(cone)
            actual_num_variables = num_variables - len(dimensions_removed)
    
            needed_points = factorial(actual_num_variables + max_degree)//( factorial(actual_num_variables)*factorial(max_degree) )
            cone_points = generate_cone_points(cone, needed_points)
            term_dict = get_term_dict(A_matrix, cone_points)

            interpolation_variables = [x for dim, x in enumerate(x_vars) if dim not in dimensions_removed]
            T = PolynomialRing(QQ, names=interpolation_variables)
            cone_points = [[p for dim, p in enumerate(point) if dim not in dimensions_removed] for point in cone_points]

            cone_poly = 0
            for degree, terms in term_dict.items():
                cone_poly += T.interpolation(max_degree, cone_points, terms)*k**degree

            cone_polys.append(cone_poly)
    return cone_polys
        

In [53]:
print(sec_fan.cones())
cone_polys = _compute_piecewise(A, sec_fan)
cone_polys

{2: [[]], 3: [[0], [1], [2]], 4: [[0, 1], [1, 2]]}
num_variables=4


[(9/2*x2^2 + 3*x2*x3 + 1/2*x3^2)*k^2 + (9/2*x2 + 3/2*x3)*k + 1,
 (5/2*x2^2 + 5*x2*x3)*k^2 + (7/2*x2 + 2*x3)*k + 1]

In [56]:
cones = []
rays = [free_module_element(ray) for ray in  sec_fan.rays()]
for ray_list in sec_fan.maximal_cones().values():
    for ray_indeces in ray_list:
        if ray_indeces != []:
            cones.append(Cone([rays[idx] for idx in ray_indeces]))
cones

[2-d cone in 4-d lattice N, 2-d cone in 4-d lattice N]

In [59]:
for p in rays:
    print(f"point: {p}")
    vertices = create_polytope_from_matrix(A, p).Vrepresentation()
    ehr_poly = ehrhart_quasi_polynomial(vertices)
    print(f"{ehr_poly=}")

    for idx, cone in enumerate(cones):
        if p in cone:
            print(f"cone idx: {idx}")
            print(cone_polys[idx+1](x0=p[0], x1=p[1], x2=p[2], x3=p[3]))
        continue
    print()

point: (-1, 2, -1, 3)
ehr_poly=QuasiPolynomialElement(Ring of Quasi-Polynomials over Rational Field, [[1]])
cone idx: 0
-25/2*k^2 + 5/2*k + 1

point: (1, 3, 1, 2)
ehr_poly=QuasiPolynomialElement(Ring of Quasi-Polynomials over Rational Field, [[1], [15/2], [25/2]])
cone idx: 0
25/2*k^2 + 15/2*k + 1
cone idx: 1


IndexError: list index out of range

In [ ]:
from random import randint
random_points = [sum(ray*randint(0, 3) for ray in rays) for k in range(10)]
random_points

In [ ]:
for p in random_points:
    print(f"point: {p}")
    vertices = create_polytope_from_matrix(A, p).Vrepresentation()
    ehr_poly = ehrhart_quasi_polynomial(vertices)
    print(f"{ehr_poly=}")

    for idx, cone in enumerate(cones):
        if p in cone:
            print(f"cone idx: {idx}")
            print(cone_polys[idx+1](x0=p[0], x1=p[1], x2=p[2], x3=p[3]))
        continue
    print()

In [ ]:
from collections.abc import Iterable
def interpolation(self, bound, *args):
        # get ring and number of variables
        R = self.base_ring()
        n = self.ngens()

        # we only run the algorithm over fields
        if not R.is_field():
            raise TypeError(f'The base ring {R} is not a field.')

        # helper function to sample "num_samples" elements from R
        def sample_points(num_samples):
            try:
                samples = list(itertools.islice(R, num_samples))
                if len(samples) < num_samples:
                    raise ValueError(f'Could not sample {num_samples} different elements of {R}.')
            except NotImplementedError:
                if R.characteristic() == 0 or R.characteristic() >= num_samples:
                    samples = [R(k) for k in range(num_samples)]
                else:
                    raise NotImplementedError(f'Could not sample {num_samples} different elements of {R}.')

            return samples

        # set points and values
        if len(args) == 2:
            points, values = args
        else:
            F, = args

            if isinstance(bound, Iterable):
                R_points = sample_points(max(bound) + 1)
                points = list(itertools.product(*[R_points[:bound[i] + 1] for i in range(n)]))
            else:
                points = list(itertools.combinations_with_replacement(sample_points(bound + 1), n))

            values = [F(*x) for x in points]

        # find all possibly appearing exponents
        if isinstance(bound, Iterable):
            exponents_space = list(itertools.product(*(range(bound[i] + 1) for i in range(n))))
        else:
            exponents_space = []
            for entry in itertools.combinations_with_replacement(range(bound + 1), n):
                exponents_space.append([entry[0]] + [entry[i] - entry[i - 1] for i in range(1, n)])
    
        # build matrix
        M = matrix.zero(R, 0, len(points))
        for exponents in exponents_space:
            M = M.stack(vector(R, [self.monomial(*exponents)(*x) for x in points]))

        # solve for coefficients and construct polynomial
        try:
            coeff = M.solve_left(vector(R, values))
        except ValueError:
            raise ValueError('Could not find a solution.')
        solution = sum(coeff[i] * self.monomial(*exponents_space[i]) for i in range(len(exponents_space)))

        return solution